In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
df=pd.read_csv("Clear_Path_300ms_delay01_subt2.csv",sep="	")
c = 3e+8;
#computing the frequencies
freq_array = np.array((c*1e+9)/df["Wavelength(nm)"])
df["Frequency (s^(-1))"] = freq_array
new_freq_array = np.round(freq_array / 1e+13, decimals = 0)
#labelling the frequency values by creating classes 
y = LabelEncoder().fit_transform(new_freq_array)
X = np.reshape(freq_array, (2068, 1))
#creating a dictionary for sampling_strategy
counter = Counter(y)
for k,v in counter.items():
 per = v / len(y) * 100
 print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=4, n=137 (6.625%)
Class=3, n=461 (22.292%)
Class=2, n=547 (26.451%)
Class=1, n=681 (32.930%)
Class=0, n=242 (11.702%)


In [4]:
values=np.empty(5); values.fill(700)
strategy = dict(list(enumerate(values)))
oversample = SMOTE(sampling_strategy = "all", random_state = 50)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)
a = X_over.flatten()
b = np.sort(a)

In [9]:
eq_freq_array = np.linspace(b[0], b[3404], 6204)
interp = interpolate.interp1d(df["Frequency (s^(-1))"],df["Intensity"],kind="cubic")
new_intensity = interp(eq_freq_array)
new_intensity

array([-1.78      , -3.77142703, -0.30286587, ...,  1.7011026 ,
       23.48608668, 58.62      ])

In [20]:
interp = interpolate.interp1d(df["Frequency (s^(-1))"],df["Intensity"],kind="cubic")
new_intensity = interp(eq_freq_array)
data = {"Frequency" : eq_freq_array, "Intensity" : new_intensity}
df_new = pd.DataFrame(data)

In [22]:
from scipy.fft import fft, ifft
Y=ifft(np.array(df_new["Intensity"]))
#calculating the phase shift caused by dispersion
dispersion_phase=np.angle(Y,deg=False)
df_new["dispersion_phase"]=dispersion_phase
df_new

,Frequency,Intensity,dispersion_phase
0,3.520482e+14,-1.780000,-0.000000
1,3.520541e+14,-3.771427,-2.268495
2,3.520599e+14,-0.302866,1.743027
3,3.520657e+14,2.465708,2.886793
4,3.520716e+14,-1.625363,0.612903
...,...,...,...
6199,3.882392e+14,-12.436839,-1.064411
6200,3.882450e+14,-9.388643,-0.612903
6201,3.882509e+14,1.701103,-2.886793
6202,3.882567e+14,23.486087,-1.743027
